# **Train**

We want to predict the change of the price of crypto a few minutes in the future based on the prices of past minutes. <br>
The dataset is generated with binance-api. Code for dataset generation will be on github soon.<br>
(https://github.com/dercodeKoenig/crypto-forecast-jupyterlab)<br>
Data looks like this:<br>
[<br>
    &nbsp;&nbsp;&nbsp;[date, time, open, high, low, close, volume],<br>
    &nbsp;&nbsp;&nbsp;[date, time, open, high, low, close, volume],<br>
    &nbsp;&nbsp;&nbsp;...<br>
    &nbsp;&nbsp;&nbsp;[date, time, open, high, low, close, volume],<br>
 ]<br>
There is one entry for each minute<br>

Lets put it into a neural network and start trading. $$$

In [ ]:
from IPython.display import clear_output
import numpy as np
import tensorflow as tf
import copy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

I uploaded a dataset on kaggle. It contains all Prices for the listed Symbols.<br>
We only need to choose one.

In [ ]:
datadir = "../input/ethusdt01012018/"

valid_symbols = ['ETHUSDT', 'AAVEUSDT', 'ADAUSDT', 'ADXUSDT', 'AIONUSDT', 'AVAXUSDT', 'AXSUSDT', 'BATUSDT', 'BLZUSDT', 'BNBUSDT', 'BNTUSDT', 'CHRUSDT', 'CVCUSDT', 'DASHUSDT', 'DATAUSDT', 'DENTUSDT', 'DEXEUSDT', 'DOTUSDT', 'ELFUSDT', 'ENJUSDT', 'EOSUSDT', 'ETCUSDT', 'FIROUSDT', 'FTMUSDT', 'FUNUSDT', 'GALAUSDT', 'GHSTUSDT', 'GRTUSDT', 'GXSUSDT', 'HOTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'KEYUSDT', 'KMDUSDT', 'KNCUSDT', 'LINKUSDT', 'LRCUSDT', 'LSKUSDT', 'LTCUSDT', 'MANAUSDT', 'MATICUSDT', 'MFTUSDT', 'MTLUSDT', 'NANOUSDT', 'NEOUSDT', 'OMGUSDT', 'ONTUSDT', 'POWRUSDT', 'PUNDIXUSDT', 'REPUSDT', 'RLCUSDT', 'SANDUSDT', 'SCUSDT', 'SOLUSDT', 'STRAXUSDT', 'THETAUSDT', 'TRXUSDT', 'VETUSDT', 'VGXUSDT', 'WANUSDT', 'WAVESUSDT', 'XEMUSDT', 'XLMUSDT', 'XMRUSDT', 'XRPUSDT', 'XVGUSDT', 'ZECUSDT', 'ZENUSDT', 'ZILUSDT', 'ZRXUSDT']

print(len(valid_symbols),"symbols found")
symbol = valid_symbols[0]
print("woking on", symbol)

# How does our Model work?
We will use a combination of cnn and rnn. First we do some conv1d, appending some GRUs and finally we get the predictions in dense layers.<br>
The input for the model will look like this:<br>

batches[<br>
&nbsp;&nbsp;&nbsp;&nbsp;samples[<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;[<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;samples[<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[feature1, feature2, feature3 ...]<br>
&nbsp;&nbsp;&nbsp;&nbsp;[<br>
]<br>

In this batch we have 2 objects with 4 samples for each object. This would be the past 4 minutes to predict the next price change.<br><br>
Only 4 minutes will not be enough to make acceptable predictions so we will make the sequence length a bit longer. Lets say we want the model to look for the past 3 days:<br>
seq_len = 3days * 24 hours per day * 60 minutes per hour<br>

batch size is the number of training objects to analyse before performing gradient descent<br>
looks_in_future is the number of minutes we want to look into the future.<br>

# What do we predict?
What the model will predict is not the absolute price, it is the change of the price. This is because neurons cant give us a useable number like 3000$, but they can give us a change in price, like 3% price increase in the next 50 minutes<br><br>

# More on the input data:
We will input 7 features into the model:<br>
* open price change = the change of the price where the minute started
* high price change = the change of the maximum price reached in this minute
* low price change = the change of the lowest price reached in this minute
* close price change = the change of the price at the end of this minute
* volume change = the change of the volume traded (i think it should be this)
* relative to ath max = maximum price reached this minute relative to the last all-time-high
* relative to ath min = minumum price reached this minute relative to the last all-time-high
<br><br>
lets say we have a sequence len of 2 and the last ath was 50: we need 3 entrys<br>
<table>
  <tr>
    <th>date</th>
    <th>time</th>
    <th>open</th>
    <th>high</th>
    <th>low</th>
    <th>close</th>
    <th>volume</th>
  </tr>
  <tr>
    <td>1.1.18</td>
    <td>00:01:00</td>
    <td>30</td>
    <td>35</td>
    <td>28</td>
    <td>29</td>
    <td>100</td>
  </tr>
    <tr>
    <td>1.1.18</td>
    <td>00:02:00</td>
    <td>29</td>
    <td>30</td>
    <td>25</td>
    <td>26</td>
    <td>120</td>
  </tr>
  <tr>
    <td>1.1.18</td>
    <td>00:03:00</td>
    <td>26</td>
    <td>30</td>
    <td>26</td>
    <td>29</td>
    <td>80</td>
  </tr>
</table>
our input sequence should be like this:
<table>
  <tr>
    <th>open_change</th>
    <th>high_change</th>
    <th>low_change</th>
    <th>close_change</th>
    <th>volume_change</th>
    <th>price_high_relative</th>
    <th>price_low_relative</th>
  </tr>
   <tr>
    <td>-0.03</td>
    <td>-0.14</td>
    <td>-0.1</td>
    <td>-0.1</td>
    <td>0.2</td>
    <td>0.6</td>
    <td>0.5</td>
  </tr>
   <tr>
    <td>-0.1</td>
    <td>0</td>
    <td>0.04</td>
    <td>0.1</td>
    <td>-0.16</td>
    <td>0.6</td>
    <td>0.52</td>
  </tr>
 
</table>

# The Output:
for the specified minutes to forecast we will predict the highest and the lowest value to be reached relative to the last sample in the input sequence

In [ ]:
seq_len = int(24*3*60)
batch_size = 32
learnrate = 0.000005
looks_in_future = 20

In [ ]:
#load the data
npd = np.load(datadir+symbol+".npy",allow_pickle=True)
print("loaded:", npd.shape)

In [ ]:
#for each entry we are calculating the maximum price this crypto reached until than
all_time_high = []
last_max_val = 0
for i in tqdm(range(1,npd.shape[0])):
    current_val = float(npd[i][3])
    if(current_val > last_max_val):
        last_max_val = current_val
    
    all_time_high.append(last_max_val)

print("")
plt.title("all-time-high values for each entry")
plt.plot(all_time_high)
plt.show()

In [ ]:
npd_relative = []
print("converting to relative data...")

prices_relative_to_ath_max = []
prices_relative_to_ath_min = []

relative_price_changes_max = []
relative_price_changes_min = []

for i in tqdm(range(npd.shape[0]-1)):
    #changes, append *10 for numbers to note be too small
    price_open  = (float(npd[i+1][2])-float(npd[i][2]))/float(npd[i][2])*10
    price_high  = (float(npd[i+1][3])-float(npd[i][3]))/float(npd[i][3])*10
    price_low   = (float(npd[i+1][4])-float(npd[i][4]))/float(npd[i][4])*10
    price_close = (float(npd[i+1][5])-float(npd[i][5]))/float(npd[i][5])*10
    volume = 0
    if float(npd[i][6]) > 0:
        volume = (float(npd[i+1][6])-float(npd[i][6]))/float(npd[i][6])
        
    #max/min price in price / ath
    price_relative_ath_max = float(npd[i+1][3]) / all_time_high[i]
    price_relative_ath_min = float(npd[i+1][4]) / all_time_high[i]
    
    prices_relative_to_ath_max.append(price_relative_ath_max)
    prices_relative_to_ath_min.append(price_relative_ath_min)
    relative_price_changes_min.append(price_high)
    relative_price_changes_max.append(price_low)
    
    npd_relative.append((price_open, price_high, price_low, price_close, volume, price_relative_ath_max, price_relative_ath_min))
    
npd = np.array(npd_relative,dtype="float32")

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1, figsize=(15,5))
ax1.set_title("relative high/low values")
ax1.plot(prices_relative_to_ath_min, color="red")
ax1.plot(prices_relative_to_ath_max, color="green")

ax2.set_title("changes in price (high/low)")
ax2.plot(relative_price_changes_min, color="red")
ax2.plot(relative_price_changes_max, color="green")

fig.show()

In [ ]:
def unpack_batch(batch):
    X=[]
    Y=[]
    for i in batch:
            X.append(i[0])
            Y.append(i[1])
    X = np.array(X,dtype="float32")
    Y = np.array(Y,dtype="float32")
    
    return (X, Y)
          
def array_to_samples(ar):
    samples = []
    l = ar.shape[0]-seq_len-looks_in_future
    
    for i in range(l):
        array_future_data = ar[i+seq_len+1:i+seq_len+1+looks_in_future]
        maxval = np.amax(array_future_data, axis=0)[5]
        minval = np.amin(array_future_data, axis=0)[6]
        
        # again *10 for same reason
        maxval_relative = (maxval - ar[i+seq_len][5]) /ar[i+seq_len][5]*10
        minval_relative = (minval - ar[i+seq_len][6]) /ar[i+seq_len][6]*10
        
        data = (ar[i:i+seq_len],(maxval_relative,minval_relative))
        samples.append(np.array(data,dtype=object))
    
    return np.array(samples,dtype=object)

In [ ]:
samples_len = npd.shape[0] - seq_len - looks_in_future
batch_num = int(samples_len / batch_size)
print("number of batches:",batch_num)

In [ ]:
l_in = tf.keras.layers.Input(shape=(seq_len,7), batch_size=batch_size)

c1 = tf.keras.layers.Conv1D(64,16,activation="relu")(l_in)
p1 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(c1)

c2 = tf.keras.layers.Conv1D(128,8,activation="relu")(p1)
p2 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(c2)

c3 = tf.keras.layers.Conv1D(256,4,activation="relu")(p2)
p3 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(c3)

c4 = tf.keras.layers.Conv1D(512,2,activation="relu")(p3)
p4 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(c4)

c5 = tf.keras.layers.Conv1D(32,1,activation="relu")(p4)
p5 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(c5)

rn1 = tf.keras.layers.LSTM(512, return_sequences=False)(p5)

flat = tf.keras.layers.Flatten()(rn1)
d1 = tf.keras.layers.Dense(512, activation="tanh")(flat)
d2 = tf.keras.layers.Dense(256, activation="tanh")(d1)
l_out = tf.keras.layers.Dense(2, activation="tanh")(d2)

model = tf.keras.Model(l_in, l_out)
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learnrate)

losses = []
ct = 0


y_true_vals = []
y_output_vals = []

t0 = time.time()
for s in range(batch_num):
    curr_batch = npd[s * batch_size: (s + 1) * batch_size + seq_len + looks_in_future]
    samples = array_to_samples(curr_batch)
    X, Y = unpack_batch(samples)

    with tf.GradientTape() as tape:
        outputs = model(X)
        loss = tf.keras.losses.mse(Y,outputs)
    
    
    gradients = tape.gradient(loss,model.trainable_variables)
    opt.apply_gradients(zip(gradients,model.trainable_variables))
   
    for i in Y:
        y_true_vals.append(i[0])
    for i in outputs:
        y_output_vals.append(i[0])
    losses.append(tf.nn.compute_average_loss(loss).numpy())
    
    ct+=1
    if(ct%100==0):
        clear_output()
        plt.figure(figsize=(22,4))
        plt.plot(losses)
        plt.show()
        
        plt.figure(figsize=(22,4))
        plt.plot(y_true_vals, color = "green")
        plt.plot(y_output_vals, color = "red")
        plt.show()
        
    t1 = time.time()
    past = t1-t0
    t0 = t1
    print("\r",s,"/",batch_num, ":", str(past)+"s   ("+str((batch_num-s)*past)+"s left)", end="")

        
clear_output()

In [ ]:
plt.figure(figsize=(22,4))
_=plt.plot(losses)
plt.show()

plt.figure(figsize=(22,4))
_=plt.plot(y_true_vals, color = "green")
_=plt.plot(y_output_vals, color = "red")
plt.show()


model.save_weights(symbol)

np.save("y_true_vals",np.array(y_true_vals))
np.save("y_output_vals",np.array(y_output_vals))
np.save("losses",np.array(losses))
